In [86]:
from pprint import pprint
import pandas as pd
import numpy as np
import random
import sys
rng = np.random.default_rng(2022)
RECIPIENT_PRO_REVEAL = 52.9
RECIPIENT_CONTRA_REVEAL = 52
PROPORTIONS = 70.92
MODE_DG_DECISION = 0
df = pd.read_csv('./calcs/polar_2022-06-02.csv')
b_r_assns = pd.read_csv('/Users/philippchapkovski/Documents/war_polarization_data/assignments_baseline_r.csv')
# values = set(b_r_assns['IDs'])
baseline_r_session = '88y49odh'
fr_d_session = '7az6ulp9'
rb_d_session = '9ig1b5o7'
dictator_sessions = [fr_d_session, rb_d_session]

baseline_r_df = df[(df['session.code'] == baseline_r_session) & (df['player.payable'] == True)]
baseline_r_df = baseline_r_df[df['participant.label'].isin(b_r_assns['participant_label'])]
ddf = df[(df['session.code'].isin(dictator_sessions)) & (df['player.payable'] == True)]
pprint(ddf.shape)

#
#
def belief_calc(i):
    if i['belief_column_name'] == 'player.own_dg_belief':
        return abs(i['belief_value'] - i['player.dg_decision'])
    if i['belief_column_name'] == 'player.vl_pro_belief':
        return abs(i['belief_value'] - RECIPIENT_PRO_REVEAL)
    if i['belief_column_name'] == 'player.average_dg_belief':
        return abs(i['belief_value'] - MODE_DG_DECISION)
    if i['belief_column_name'] == 'player.vl_contra_belief':
        return abs(i['belief_value'] - RECIPIENT_CONTRA_REVEAL)
    if i['belief_column_name'] == 'player.proportion':
        return abs(i['belief_value'] - PROPORTIONS)
    #
    return 100
#

# #  let's calculate payoffs of dictators
d_beliefs = ['player.proportion', 'player.average_dg_belief']
selected_columns_d = ['participant.label', 'player.dg_decision', 'player.partner_position'] + d_beliefs
ddf = ddf[selected_columns_d]
ddf['rpayoff'] = 100 - ddf['player.dg_decision']
ddf['belief_column_name'] = rng.choice(d_beliefs, len(ddf), replace=True)
ddf['belief_value'] = ddf.to_numpy()[np.arange(ddf.shape[0]), ddf.columns.get_indexer(ddf['belief_column_name'])]
ddf['belief_match'] = ddf.apply(belief_calc, 1)

ddf['get_belief_payoff'] = ddf['belief_match'] < 11
ddf['belief_payoff'] = ddf['get_belief_payoff'] * 0.25
ddf['total_payoff'] = round(ddf['belief_payoff'] + ddf['rpayoff'] / 100, 2)

ddf.to_csv('calcs/fd_rb_d_payoffs.csv',index=False)
pprint(ddf.total_payoff.sum())

# # Let's calculate payoffs of receivers

# we create a slightly reshaped df of dictators so we can match them
ddf['full_partner_pos'] = ddf['player.partner_position']
ddf.full_partner_pos = ddf.full_partner_pos.fillna('nr')

ddf.loc[ddf['full_partner_pos']=='no', 'for_matching'] = 'no'
ddf.loc[ddf['full_partner_pos']=='yes', 'for_matching'] = 'yes'

ddf.loc[ddf[ddf['full_partner_pos']=='nr'].index[0:99],'for_matching']='no'
ddf.loc[ddf[ddf['full_partner_pos']=='nr'].index[99:],'for_matching']='yes'
ddf.groupby('for_matching')['for_matching'].count()
ddf = ddf.iloc[:599]


ddf.sort_values(by='for_matching', inplace=True)
ddf['row_num'] = np.arange(len(ddf))
ddf.set_index('row_num', inplace=True)
ddf.sort_index(inplace=True)
#
ddf.rename(columns={'participant.label':'dictator_label'}, inplace=True)
rrr_beliefs = ['player.proportion', 'player.own_dg_belief', ]
#
selected_columns_r = ['participant.label',
                      'player.opinion_war']  + rrr_beliefs
#
baseline_r_df = baseline_r_df[selected_columns_r]
baseline_r_df['own_position'] =baseline_r_df['player.opinion_war']
rdf = baseline_r_df
rdf.sort_values(by='own_position', inplace=True)
rdf['row_num'] = np.arange(len(rdf))
rdf.set_index('row_num', inplace=True)
rdf.sort_index(inplace=True)
ddf = ddf[['player.dg_decision', 'dictator_label']]
#
rdf=rdf.join(ddf)
rdf['rpayoff'] = 50+rdf['player.dg_decision']
rdf['belief_column_name'] = rng.choice(rrr_beliefs, len(rdf), replace=True)
rdf['belief_value'] = rdf.to_numpy()[np.arange(rdf.shape[0]),rdf.columns.get_indexer(rdf.belief_column_name)]
rdf['belief_match']= rdf.apply(belief_calc, 1)
rdf['get_belief_payoff']=  rdf['belief_match']<11
rdf['belief_payoff']=  rdf['get_belief_payoff']*0.25
rdf['total_payoff'] = rdf['belief_payoff'] +rdf['rpayoff']/100
print(rdf.total_payoff.sum())
rdf.to_csv('calcs/baseline_r_payoffs.csv',index=False)
#
#
#
#





(601, 89)
709.95
318.8


/var/folders/xr/m4_30mss4bn_vqnkrhn87tdw0000gn/T/ipykernel_51379/840582294.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  baseline_r_df = baseline_r_df[df['participant.label'].isin(b_r_assns['participant_label'])]
